In [1]:
import json
import pandas as pd
import re

df = pd.read_csv("../../No3Project/M1data.csv",encoding="utf-8")
df["phone"] = df["phone"].apply(lambda x : "0" + str(x))



In [2]:
import jieba
jieba.load_userdict("../../No3Project/dict.txt")
df

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\BIGDAT~1\AppData\Local\Temp\jieba.cache
Loading model cost 0.759 seconds.
Prefix dict has been built succesfully.


,article_hit,author,comment,content,date,href,img,info,rate,species,title,phone,Name
0,41次,bbone,[],\r\r\r\n地址: 台北市中山區中山北路二段20巷1-1號\r\r\r\n電話: 02...,2018-06-20,https://www.mobile01.com/waypointdetail.php?id...,['https://attach.mobile01.com/attach/201806/th...,"[{'rate': 3, 'place': '台北市中山區中山北路二段20巷1-1號', '...",3,餐飲,台北中山。月半そば蕎麥麵,0225638008,二月半そば蕎麥麵
1,60次,allyyang701,['takashi1730:介紹的很文青但看價格與份量，應該是普遍台北巷弄美食的生態了器皿也...,文章出處Ally愛玩樂 \r\r\r\n\r\r\r\n \r\r\r\nJana Ca...,2018-06-17,https://www.mobile01.com/waypointdetail.php?id...,['https://attach.mobile01.com/attach/201806/th...,"[{'rate': 5, 'place': '台北市大安區泰順街60巷27號1樓右側', '...",5,餐飲,【台北市 大安區/捷運台電大樓站】Jana Cafe restaurant 嚼咖啡餐廳-沈...,0223685345,Jana cafe嚼咖啡餐廳
2,47次,老狗5550,"['蜜嚕熊:這間真的是經典從阿公時代就吃的餐廳!!推', 'axr6902421:10幾年前...","波麗路西餐廳是臺灣現今存在最老的西餐廳，\r\r\r\n招牌上如果掛上""臺灣第一家""，\r\...",2018-05-24,https://www.mobile01.com/waypointdetail.php?id...,['//attach.mobile01.com/waypoint/mobile01-4d61...,"[{'rate': 5, 'place': 'Bolero波麗路西餐廳', 'phone':...",5,餐飲,[食記] 臺北市 大同區 波麗路西餐分店 -- 哥和姊吃的不止是歷史，更是好滋味,0225560710,BOLERO 波麗路西餐廳
3,238次,Prettyflower,[],\r\r\r\n地址:台北市松山區富錦街469號\r\r\r\n電話: 02-276784...,2018-05-10,https://www.mobile01.com/waypointdetail.php?id...,['https://attach.mobile01.com/attach/201805/th...,"[{'rate': 4, 'place': '台北市松山區富錦街469號', 'phone'...",4,餐飲,台北市松山區富錦街綠色隧道+哈古小館法式下午茶~ 氛圍佳~有質感~,0227678483,哈古小館 au petit cochon
4,131次,gochi2100,[],\r\r\r\n \r\r\r\n這是mobile01抽獎免費試吃文。\r\r\r\n \...,2018-05-06,https://www.mobile01.com/waypointdetail.php?id...,['https://attach.mobile01.com/attach/201805/th...,"[{'rate': 5, 'place': '台北市錦州街200號', 'phone': '...",5,餐飲,(食)台北 風來坊串串錦店 銷魂名古屋手羽先,0225425882,風來坊串串(錦店)
5,596次,allyyang701,[],文章出處 \r\r\r\n\r\r\r\n \r\r\r\n\r\r\r\n電話：（02)2...,2018-04-09,https://www.mobile01.com/waypointdetail.php?id...,['https://attach.mobile01.com/attach/201804/th...,"[{'rate': 4, 'place': '福國路59號', 'phone': '（02)...",4,餐飲,【台北市 士林區/捷運芝山站】Moda Vita Caffè-從早午餐到甜點都愛不釋口,0228319996,Moda Vita Caffe
6,9次,allyyang701,['星空羽:感謝分享！內容十分詳細，店內的裝潢風格看起來不錯！食物也看起來不錯！'],文章出處Ally愛玩樂\r\r\r\n\r\r\r\n\r\r\r\n櫻花羿日式炭火燒肉\...,2018-09-12,https://www.mobile01.com/waypointdetail.php?id...,['https://attach.mobile01.com/attach/201809/th...,"[{'rate': 4, 'place': '大安區敦化南路一段160巷13號15號', '...",4,餐飲,【台北市 大安區/捷運忠孝復興站】櫻花羿日式炭火燒肉-東區CP值高的燒肉吃到飽,0227789222,櫻花羿日式精緻炭火燒肉
7,40次,老狗5550,[],老狗住在文山區，最痛苦的是想吃餐牛排，總得東找西找，才能避免踩到雷。\r\r\r\n文山區最...,2018-09-09,https://www.mobile01.com/waypointdetail.php?id...,['https://attach.mobile01.com/attach/201809/mo...,"[{'rate': 4, 'place': '台北市文山區景文街137號', 'phone'...",4,餐飲,[食記] 臺北市 文山區 人从众牛排 -- 夜市中的厚切牛排,0229351177,人从众厚切牛排館(景美店)
8,135次,blackfriday13,[],\r\r\r\n\r\r\r\n在京鼎樓用餐彷彿置身日本，這樣的說法一點也不誇張。而京鼎樓的...,2018-09-05,https://www.mobile01.com/waypointdetail.php?id...,['//attach.mobile01.com/waypoint/mobile01-f77c...,"[{'rate': 4, 'place': '台北市中山區長春路47號', 'phone':...",4,餐飲,【台北】京鼎樓獨家烏龍茶小籠包，被日本客攻陷，彷彿置身日本！,0225236639,京鼎樓
9,120次,老狗5550,[],老狗在木柵住了二十多年，感到最不方便的，是大台北地區很難想像的外食。家附近走路5分鐘可到的外...,2018-09-03,https://www.mobile01.com/waypointdetail.php?id...,['https://attach.mobile01.com/attach/201809/mo...,"[{'rate': 3, 'place': '台北市文山區木柵路三段15號', 'phone...",3,餐飲,[食記] 臺北市 文山區 孫東寶牛排 文山木柵店 -- 住家附近的外食新選擇,0222348886,孫東寶台式牛排教父(文山木柵店)


In [3]:
df1 = df[["content", "rate"]]
df1

,content,rate
0,\r\r\r\n地址: 台北市中山區中山北路二段20巷1-1號\r\r\r\n電話: 02...,3
1,文章出處Ally愛玩樂 \r\r\r\n\r\r\r\n \r\r\r\nJana Ca...,5
2,"波麗路西餐廳是臺灣現今存在最老的西餐廳，\r\r\r\n招牌上如果掛上""臺灣第一家""，\r\...",5
3,\r\r\r\n地址:台北市松山區富錦街469號\r\r\r\n電話: 02-276784...,4
4,\r\r\r\n \r\r\r\n這是mobile01抽獎免費試吃文。\r\r\r\n \...,5
5,文章出處 \r\r\r\n\r\r\r\n \r\r\r\n\r\r\r\n電話：（02)2...,4
6,文章出處Ally愛玩樂\r\r\r\n\r\r\r\n\r\r\r\n櫻花羿日式炭火燒肉\...,4
7,老狗住在文山區，最痛苦的是想吃餐牛排，總得東找西找，才能避免踩到雷。\r\r\r\n文山區最...,4
8,\r\r\r\n\r\r\r\n在京鼎樓用餐彷彿置身日本，這樣的說法一點也不誇張。而京鼎樓的...,4
9,老狗在木柵住了二十多年，感到最不方便的，是大台北地區很難想像的外食。家附近走路5分鐘可到的外...,3


In [5]:
# positive = 1 negative = 0 
df1["NLP_rate"] = df["rate"].apply(lambda x : 1 if x >= 4 else ( 0 if x < 3 else "UNSUP"))
df1

C:\Users\Big data\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,content,rate,NLP_rate
0,\r\r\r\n地址: 台北市中山區中山北路二段20巷1-1號\r\r\r\n電話: 02...,3,UNSUP
1,文章出處Ally愛玩樂 \r\r\r\n\r\r\r\n \r\r\r\nJana Ca...,5,1
2,"波麗路西餐廳是臺灣現今存在最老的西餐廳，\r\r\r\n招牌上如果掛上""臺灣第一家""，\r\...",5,1
3,\r\r\r\n地址:台北市松山區富錦街469號\r\r\r\n電話: 02-276784...,4,1
4,\r\r\r\n \r\r\r\n這是mobile01抽獎免費試吃文。\r\r\r\n \...,5,1
5,文章出處 \r\r\r\n\r\r\r\n \r\r\r\n\r\r\r\n電話：（02)2...,4,1
6,文章出處Ally愛玩樂\r\r\r\n\r\r\r\n\r\r\r\n櫻花羿日式炭火燒肉\...,4,1
7,老狗住在文山區，最痛苦的是想吃餐牛排，總得東找西找，才能避免踩到雷。\r\r\r\n文山區最...,4,1
8,\r\r\r\n\r\r\r\n在京鼎樓用餐彷彿置身日本，這樣的說法一點也不誇張。而京鼎樓的...,4,1
9,老狗在木柵住了二十多年，感到最不方便的，是大台北地區很難想像的外食。家附近走路5分鐘可到的外...,3,UNSUP


In [6]:
import re
def clean_text(x):
    x = re.sub(r"\W+","",x)
    words = jieba.cut(x,cut_all=False)
    return " ".join(words)


df1["clean"] = df1["content"].apply(clean_text)
df1

C:\Users\Big data\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,content,rate,NLP_rate,clean
0,\r\r\r\n地址: 台北市中山區中山北路二段20巷1-1號\r\r\r\n電話: 02...,3,UNSUP,地址 台北市 中山區 中山北路 二段 20 巷 11 號 電話 0 225 638008 營...
1,文章出處Ally愛玩樂 \r\r\r\n\r\r\r\n \r\r\r\nJana Ca...,5,1,文章 出處 Ally 愛玩 樂 JanaCaferestaurant 嚼 咖啡 餐廳 電話 ...
2,"波麗路西餐廳是臺灣現今存在最老的西餐廳，\r\r\r\n招牌上如果掛上""臺灣第一家""，\r\...",5,1,波麗 路西 餐廳 是 臺灣 現今 存在 最老 的 西餐廳 招牌 上 如果 掛上 臺灣 第一家...
3,\r\r\r\n地址:台北市松山區富錦街469號\r\r\r\n電話: 02-276784...,4,1,地址 台北市 松山區 富錦 街 469 號 電話 0227678483 營業時間 0830 ...
4,\r\r\r\n \r\r\r\n這是mobile01抽獎免費試吃文。\r\r\r\n \...,5,1,這是 mobile01 抽獎 免費 試吃 文 這次 的 獎項 是 胡麻 松阪 沙拉 手羽 先...
5,文章出處 \r\r\r\n\r\r\r\n \r\r\r\n\r\r\r\n電話：（02)2...,4,1,文章 出處 電話 0 228 319996 地址 台北市 士林區 福國路 59 號 交通 捷...
6,文章出處Ally愛玩樂\r\r\r\n\r\r\r\n\r\r\r\n櫻花羿日式炭火燒肉\...,4,1,文章 出處 Ally 愛玩 樂 櫻花 羿 日式 炭火 燒肉 電話 0227789222 地址...
7,老狗住在文山區，最痛苦的是想吃餐牛排，總得東找西找，才能避免踩到雷。\r\r\r\n文山區最...,4,1,老狗 住在 文山區 最痛苦 的是 想吃餐 牛排 總得 東找 西 找 才能 避免 踩到 雷文 ...
8,\r\r\r\n\r\r\r\n在京鼎樓用餐彷彿置身日本，這樣的說法一點也不誇張。而京鼎樓的...,4,1,在 京鼎樓 用餐 彷彿 置身 日本 這樣 的 說法 一點也不 誇張 而京鼎樓 的 招牌 烏龍...
9,老狗在木柵住了二十多年，感到最不方便的，是大台北地區很難想像的外食。家附近走路5分鐘可到的外...,3,UNSUP,老狗 在 木柵 住 了 二十多年 感到 最不 方便 的是 大台北地區 很難想像 的 外 食家...


In [116]:
import numpy as np
from sklearn.model_selection import train_test_split
df2 = df1[df1["NLP_rate"] != "UNSUP"]
df3 = df1[df1["NLP_rate"] == "UNSUP"]
unsup = df3["clean"]
# 轉換 np array


x_train,x_test,y_train,y_test = train_test_split(df2["clean"],df2["NLP_rate"],test_size=0.1)
#x_train

In [117]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from gensim.utils import to_unicode
def tag(words,label):
    word_set = []
    for i, word in enumerate(words):
        label_name = '%s_%s'%(label,i)
        word_set.append(TaggedDocument(word, [label_name]))
    
    return word_set

x_train = tag(x_train, label='TRAIN')
x_test = tag(x_test,label='TEST')
unsup = tag(unsup, label='UNSUP')



In [119]:
# 訓練 Doc2Vect
"""
model = gensim.models.doc2vec.Doc2Vec(vector_size=150, min_count=1, epochs=50, workers=7)

https://blog.csdn.net/lenbow/article/details/52120230
"""


model = Doc2Vec(dm=1, vector_size=100, min_count=1, worker=7)

model.build_vocab(x_train + x_test + unsup)

alldoc = x_train + unsup

# train

model.train(alldoc, total_examples=model.corpus_count, epochs=30)





In [120]:
model.save("../doc2vec_1.d2v")

In [132]:
model.docvecs["TRAIN_1"].reshape((1,100)).shape

(1, 100)

In [137]:
def getVec(model, corpus, size):
    vec = [model.docvecs[z.tags[0]].reshape(1,size) for z in corpus]
    return np.concatenate(vec)

x_train_vec = getVec(model, x_train, 100)
x_test_vec = getVec(model, x_test, 100)
unsup_vec = getVec(model, unsup, 100)

In [166]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

# loss = logistic penalty = L1

# cli_SGD = SGDClassifier(loss='log',penalty='l1')
y_train_np = np.array(y_train ,dtype = int )
# cli_SGD.fit(x_train_vec, y_train_np)


cli_lg = LogisticRegression()
cli_lg.fit(x_train_vec, y_train_np)




C:\Users\Big data\AppData\Local\Programs\Python\Python36\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [178]:
df.to_csv("../m1.csv", index=False)

In [187]:
m1 = df[["content","rate"]]

In [188]:
m1["content"] = m1["content"].apply(lambda x : re.sub(r"\W+","",x))
m1

C:\Users\Big data\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,content,rate
0,地址台北市中山區中山北路二段20巷11號電話0225638008營業時間1130143017...,3
1,文章出處Ally愛玩樂JanaCaferestaurant嚼咖啡餐廳電話0223685345...,5
2,波麗路西餐廳是臺灣現今存在最老的西餐廳招牌上如果掛上臺灣第一家大概沒人敢挑戰他們的權威波麗路...,5
3,地址台北市松山區富錦街469號電話0227678483營業時間08301730周二公休今日下...,4
4,這是mobile01抽獎免費試吃文這次的獎項是胡麻松阪沙拉手羽先唐揚招牌串揚8串綜合串燒4串...,5
5,文章出處電話0228319996地址台北市士林區福國路59號交通捷運芝山站1號出口步行約2分...,4
6,文章出處Ally愛玩樂櫻花羿日式炭火燒肉電話0227789222地址台北市大安區敦化南路一段...,4
7,老狗住在文山區最痛苦的是想吃餐牛排總得東找西找才能避免踩到雷文山區最熱鬧的市集景美夜市中雖然...,4
8,在京鼎樓用餐彷彿置身日本這樣的說法一點也不誇張而京鼎樓的招牌烏龍茶小籠包更是吸引許多人來嚐鮮...,4
9,老狗在木柵住了二十多年感到最不方便的是大台北地區很難想像的外食家附近走路5分鐘可到的外食商家...,3


In [189]:
m1.to_csv("../m1.csv",index=False)

In [ ]:
# def clean(x):
#     li = []
#     for words in x:
#         words = re.sub(r"\W+","",words)
#         word = jieba.cut(words,cut_all=False)
#         for w in word:
#             li.append(w)
#     return li
    
# df2["content_c"] = df2["content"].apply(clean)
# df2

In [ ]:
# df2["content_c"] = df2["content_c"].apply(lambda x : " ".join(x))
# df2

In [ ]:
# df2["rate_c"] = df2["rate"].apply(lambda x : round(x,0))
# df2

In [ ]:
# dd = df2.groupby("rate_c")["content_c"].apply(lambda x : x.tolist())

In [ ]:
# dd.index

In [ ]:
# import gensim
# ddf = pd.DataFrame({
#     "rate":dd.index,
#     "word":dd.values
# })
# ddf

In [ ]:
# from gensim.models.doc2vec import TaggedDocument

# wordset = []
# for i,words in enumerate(ddf["word"]):
#     for s,word in enumerate(words):
#         wordset.append(TaggedDocument(gensim.utils.to_unicode(word).split(),[str(i+1) + "_" + str(s)]))

# wordset

In [ ]:

# model = gensim.models.doc2vec.Doc2Vec(dm=1, vector_size=150, min_count=1, workers=7)

# model.build_vocab(wordset)


# model.train(wordset, total_examples=model.corpus_count, epochs=30)

In [ ]:
# model.save("../../model.d2v")